In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import xml.etree.ElementTree as ET
import re
import numpy as np
import nltk
import torch
from torch import nn
import transformers

from utils import *
from transformers import AutoTokenizer, BertForNextSentencePrediction
import torch
from sklearn.model_selection import train_test_split

/mnt/ssd1/code/chat-project/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import ast
test_df = pd.read_csv("synthetic_100.csv", converters={"boundaries": ast.literal_eval})

In [3]:
test_df["boundaries_mask"]

0     [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0...
1     [0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0]
2     [0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0...
3     [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1...
4     [0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0...
                            ...                        
95    [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0...
96    [0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0...
97    [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0...
98    [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0...
99    [0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0...
Name: boundaries_mask, Length: 100, dtype: object

In [4]:
model = BertForNextSentencePrediction.from_pretrained("bert-base-uncased").to("cuda:0")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model.load_state_dict(torch.load("bert_v5.bin"))
model.eval()

BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [5]:
test_df

,messages,boundaries,boundaries_mask
0,['are you good at making decisions ? '\n ' not...,"[8, 16, 20, 30]",[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0...
1,['You know that ? Joan has got the scholarship...,"[3, 8, 14, 20]",[0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0]
2,['Mary and I decided to ring the curtain down ...,"[3, 9, 17, 26]",[0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0...
3,"['Do you have girlfriend ? ' ' Yes , why ? '\n...","[16, 22, 35, 41]",[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1...
4,"['Bob , you look pale . What happened ? '\n "" ...","[6, 11, 15, 17]",[0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0...
...,...,...,...
95,"[""We've been cramming for tomorrow's history e...","[7, 15, 29, 32]",[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0...
96,"[""Great party , isn't it ? ""\n ' Yeah , really...","[9, 12, 18, 26]",[0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0...
97,"['Well , Mitzuko-san , I hope you like it here...","[9, 16, 24, 29]",[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0...
98,['Welcome to the International Business Depart...,"[7, 19, 28, 36]",[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0...


In [6]:
calculate_random_mean_windowdiff(test_df)

ValueError: Segmentations have unequal length

In [4]:
def scorer_trained_bert(messages: list[str]) -> list[float]:
    scores = []
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    device = torch.device("cuda:0")
    model.eval()

    for i in range(len(messages) - 1):
        j = i + 1

        tokenized_input = {}
        if tokenizer:
            tokenized_input = tokenizer(
                messages[i], messages[j], max_length=30, truncation=True, padding="max_length", return_tensors="pt"
            ).to(device)

        with torch.no_grad():
            logits = model.forward(**tokenized_input).logits

        score = F.softmax(logits, dim=1)[0][1]
        # print(score)
        scores.append(score.item())
        # print(score.item())
        # print(logits)
        # score = float(logits[0][0] > logits[0][1])
        # print(score)
        # scores.append(score)

    model.train()
    return scores

In [17]:
calculate_mean_windowdiff(test_df[:40], scorer=scorer_trained_bert, smoothing_strategy="depth")

0it [00:08, ?it/s]


KeyboardInterrupt: 